# VarnaAir - Open Programme

## Introduction ℹ️


## Libraries 📚
First, let's begin by importing the libraries and checking their versions.

In [32]:
import pandas
import requests

## Data provisioning 📦

In [33]:
url = "https://api.openaq.org/v3/locations"
headers = {
    "X-API-Key": "c1061fd64a9bd88562b4bb823b7900ccdfd263a9a7aa54614de834dac611a4ca"
}
params = {
    "coordinates": "43.224389,27.915733",  # center of Varna
    "radius": 5000,                        # 5 km radius
    "limit": 100,                         
}
response = requests.get(url, headers=headers, params=params)

data = response.json().get("results", [])
df_nearby_stations = pandas.DataFrame(data)
print(df_nearby_stations[["id", "name", "locality"]])

        id                         name              locality
0     8843  AMS SOU Angel Kanchev-Varna                 Варна
1  2162113           AMS Chaika - Varna  National air network


In [34]:
stations = {
    "AMS SOU Angel Kanchev-Varna": 8843, 
    "AMS Chaika - Varna": 2162113
}